## Text Network

### Install dependencies

In [42]:
import re
import itertools
import spacy
from nltk import FreqDist, bigrams
import numpy as np
import pandas as pd
import networkx as nx
from pyvis.network import Network
import seaborn as sns
from collections import Counter
sns.set(rc={'figure.figsize':(15,7)})

### Auxiliary functions

In [43]:
def remove_stopwords(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if token.is_stop == False])

def pre_processing(text):
    text = remove_stopwords(text)
    text = re.sub(r'[^\w\s]', '', text) #remove punctuation
    text = re.sub(r"\s{2,}", " ", text)
    return text

def co_occurrence(text):
    corpus = list(itertools.chain.from_iterable([text.split(' ')]))
    vocab = list(set(corpus))
    vocab_index = {word: i for i, word in enumerate(vocab)}
    bi_grams = list(bigrams(corpus))
    bigram_freq = FreqDist(bi_grams).most_common(len(bi_grams)) 
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab))) 
    
    #bigram = ((word n, word n+1), num_occurrences)
    for bigram in bigram_freq:
        current = bigram[0][1] #word n+1
        previous = bigram[0][0] #word n
        count = bigram[1] #num_occurrences
        pos_current = vocab_index[current] #obtain id for word n+1 
        pos_previous = vocab_index[previous] #obtain id for word n 
        co_occurrence_matrix[pos_current][pos_previous] = count
        
    return co_occurrence_matrix, vocab_index

def structure_text_network(matrix, vocab_index):
    data_matrix = pd.DataFrame(matrix, index=vocab_index, columns=vocab_index)
    data_stack = data_matrix.stack()
    structure = data_stack[data_stack >= 1].rename_axis(('source', 'target')).reset_index(name='weight')
    return structure[(structure.source != " ") & (structure.target != " ")]

### Generate text network

In [44]:
nlp = spacy.load("pt_core_news_sm")

In [45]:
text = """
Ministros do Supremo Tribunal Federal creem que o movimento de caminhoneiros que bloqueia estradas em diversos pontos do país é uma armadilha óbvia montada pelo entorno mais radical do presidente Jair Bolsonaro (PL), derrotado no domingo (30) por Luiz Inácio Lula da Silva (PT).
Daí a celeridade da corte em caracterizar a ilegalidade do protesto e pendurar no pescoço do governo, na figura da Polícia Rodoviária Federal, a responsabilidade pela confusão. Adiantando o debate jurídico, avaliam ministros, a posição pretendida pelo Planalto fica enfraquecida.
O plano percebido pela cúpula do Judiciário vem sendo telegrafado desde o ano passado, como exemplificou em artigo nesta Folha de S.Paulo Raul Jungmann, que na qualidade de ministro da Segurança Pública do governo Michel Temer (MDB) lidou diretamente com a greve de caminhoneiros de 2018.
Segundo a trama, o bolsonarismo estimula uma desobediência civil após a eleição de Lula e força alguma autoridade estadual a pedir a intervenção do Exército. Só que, pela Constituição, quem tem de dar tal autorização é Bolsonaro, e o impasse se coloca, culminando num embate entre Planalto e Supremo.

Ao avocar a discussão prévia, na forma da decisão de Alexandre de Moraes que já tem maioria de apoio na corte, o Supremo esvazia o controle narrativo da crise por Bolsonaro, que se manteve quieto desde que perdeu o pleito por 1,8 ponto percentual no domingo. Com efeito, Moraes já deu permissão às PMs estaduais para fazer desbloqueios em pontos que a PRF não agir, inclusive estradas federais.
A realidade joga em favor da corte, também. Bolsonaro está no momento mais isolado de todas sua meteórica carreira rumo à Presidência, conquistada em 2018. Nenhum aliado seu contestou o resultado da eleição, ao contrário: do presidente da Câmara, Arthur Lira (PP-AL), ao pastor Silas Malafaia, diversos bolsonaristas pregaram respeito às urnas.

Mais importante, o Alto-Comando do Exército fez chegar aos generais da reserva do Planalto o recado de que a Força estará ao lado da Constituição se houver algum tipo de embate institucional. Mesmo discordando com frequência e criticando ministros da corte, isso foi lido como apoiar decisões do Supremo.

Sobraram a família de Bolsonaro e os supracitados generais palacianos, todos em silêncio monástico como o presidente. Já pularam do barco o vice, general Hamilton Mourão, senador eleito, e o ministro da Casa Civil, Ciro Nogueira (PP), prócer do centrão.

Com o refluxo apontado nos protestos em alguns pontos nesta manhã de terça (1º), a jogada de Moraes parece apontar para uma resolução da crise. A ausência tática de Lula do debate é outro fator notado por observadores: o petista está evitando se envolver e dar corpo ao que é visto como um esperneio final de um presidente derrotado.

Quem terá problemas no processo parecer ser o comandante da PRF, Silvinei Vasques, que tem ameaça de prisão na ordem de Moraes. Ele já havia gerado grande desconfiança ao mobilizar operações inauditas em estradas usadas por eleitores em regiões de grande votação petista, no Nordeste, após pedir voto para o chefe em redes sociais na véspera."""


In [46]:
text_cleaned = pre_processing(text)
matrix, vocab = co_occurrence(text_cleaned)
structure_network = structure_text_network(matrix, vocab)

In [47]:
structure_network

,source,target,weight
0,,véspera,1.0
1,petista,votação,1.0
2,petista,observadores,1.0
3,ordem,prisão,1.0
4,Ministros,,1.0
...,...,...,...
263,pularam,presidente,1.0
264,telegrafado,sendo,1.0
265,usadas,estradas,1.0
266,eleitores,usadas,1.0


In [48]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])

### PageRank

In [49]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])

In [50]:
def pagerank_centrality(G):
    centrality =  nx.pagerank(G)
    return sorted(centrality.items(), key=lambda x: x[1], reverse=False)

In [51]:
centrality = pagerank_centrality(text_network)

#dicionário da centralidade
cen_dict = {}
max_cen = 0
for x in centrality:
    cen_dict[x[0]] = x[1]
    if max_cen < x[1]:
        max_cen = x[1]

### Graph Visualization

In [52]:
# lista dos nós
name_lst = [x[0] for x in centrality]

# lista de nós a serem removidos, estamos mantendo os 25 maiores
rmv_lst = name_lst[:-25]

# exibição do nome dos nós para a visualização
for i in text_network.nodes:
    text_network.nodes[i]['label'] = i

# redução do grafo mantendo a conectividade
for i in rmv_lst:
    maxi = 0
    for j in text_network.neighbors(i):
        w = name_lst.index(j)
        if maxi < w:
            maxi = w
            merge_node = j
            
    for j in text_network.neighbors(i):
        if not (j == merge_node or (j in text_network.neighbors(merge_node))):
            text_network.add_edge(merge_node, j, weight=text_network[i][j]['weight']/2)
    text_network.remove_node(i)

In [97]:
nt = Network('800px', '1000px', notebook=True)
nt.from_nx(text_network)

for edg in nt.edges:
    # espessura e física das arestas
    edg['width'] = edg['weight']**4
    if edg['weight'] < 1:
        edg['physics'] = False
        
for n in nt.nodes:
    # tamanho dos nós
    n['shape'] = 'ellipse'
    size = cen_dict[n['label']]*30/max_cen
    n['font'] = str(int(size))+'px arial white'
    n['color'] = '#7f333f'
    n['labelHighlightBold'] = True
    
    
for n in nt.nodes:
    # links nos nós
    n['title'] = "<a href='http://www.google.com\'>google <br> <a href='http://www.google.com\'>yout"
    #print(n['size'])

    
nt.show("text_network.html")

In [54]:
#print(nx.adjacency_matrix(text_network))